# **1. Informations matériel (GPU)**

In [ ]:
!nvidia-smi

Sat Mar 26 09:59:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **2. Importation des librairies**

In [ ]:
from IPython.display import Image, HTML, display
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2 
import os
import csv
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Model, load_model
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input #224*224
from keras.applications.xception import Xception, preprocess_input, decode_predictions #299*299
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import categorical_crossentropy
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical
import math
import argparse
import matplotlib
import imghdr
import pickle as pkl
import datetime
from cycler import cycler
from PIL import Image, ImageEnhance
print("Tensorflow version: "+tf.__version__)
print("Keras version: " + tf.keras.__version__)
      
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Tensorflow version: 2.8.0
Keras version: 2.8.0


#**3. Téléchargement des données d'entrainement "Small Dataset"**

In [ ]:
! wget -c http://139.162.138.92/big.tar

--2021-11-22 20:08:10--  http://139.162.138.92/big.tar
Connecting to 139.162.138.92:80... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 4149324288 (3.9G), 4149312028 (3.9G) remaining [application/octet-stream]
Saving to: ‘big.tar’

big.tar               0%[                    ]   6.29M   949KB/s    eta 1h 50m ^C


In [ ]:
! mkdir bases
! tar xf big.tar -C 'bases' --one-top-level

mkdir: cannot create directory ‘bases’: File exists
tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors


In [ ]:
del model
del train_ds
del val_ds

NameError: name 'model' is not defined

In [ ]:
import gc
gc.collect()

0

#**4. Création du fichier de labels "classes.txt"**

In [ ]:
!printf '%s\n' 'fire' 'no_fire' 'start_fire'> classes.txt
classes = ['fire', 'no_fire', 'start_fire']
num_classes = len(classes)
dataset_path = "./bases/big"

# NASNetLarge : 331
# Xception : 299
# Else : 224

input_dim = 224
batch_size = 8

# **8. Création du jeu de données d'entraînement**

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
	dataset_path,               # chemin vers le jeu de données
	validation_split=0.20,    # paritionnement du jeu de données en un jeu de validation (20%), train (80%)
	subset="training",          # Sélection du jeu de données d'entraînement
	seed=42,                    # Initialisation du générateur aléatoire (permutations)
	image_size=(input_dim,input_dim),       # Taille des images d'entrée
	batch_size=batch_size,  # Taille du mini-batch
  label_mode= 'categorical'  # Conversion au format One-Hot
)

#**9. Création du jeu de validation (A COMPLETER : même générateur, On-Hot, jeu "validation")**

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
	dataset_path,               
	validation_split=0.20,       
	subset="validation",          
	seed=42,                    
	image_size=(input_dim,input_dim),       
	batch_size=batch_size,  
  label_mode= 'categorical' 
)

Found 6003 files belonging to 3 classes.
Using 900 files for validation.


# **10. Téléchargement d'un modèle préentraîné et Transfer Learning**

In [ ]:
# Imports
import tensorflow as tf
# import tensorflow_datasets as tfds
import pandas as pd
import matplotlib.pyplot as plt
import inspect
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras import layers

In [ ]:
# List all available models
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}
print(model_dictionary)

{'DenseNet121': <function DenseNet121 at 0x7f6d46d136a8>, 'DenseNet169': <function DenseNet169 at 0x7f6d46d13730>, 'DenseNet201': <function DenseNet201 at 0x7f6d46d137b8>, 'EfficientNetB0': <function EfficientNetB0 at 0x7f6d46d13c80>, 'EfficientNetB1': <function EfficientNetB1 at 0x7f6d46d13d08>, 'EfficientNetB2': <function EfficientNetB2 at 0x7f6d46d13d90>, 'EfficientNetB3': <function EfficientNetB3 at 0x7f6d46d13e18>, 'EfficientNetB4': <function EfficientNetB4 at 0x7f6d46d13ea0>, 'EfficientNetB5': <function EfficientNetB5 at 0x7f6d46d13f28>, 'EfficientNetB6': <function EfficientNetB6 at 0x7f6d46ca2048>, 'EfficientNetB7': <function EfficientNetB7 at 0x7f6d46ca20d0>, 'InceptionResNetV2': <function InceptionResNetV2 at 0x7f6d46ca2378>, 'InceptionV3': <function InceptionV3 at 0x7f6d46ca27b8>, 'MobileNet': <function MobileNet at 0x7f6d46ca2bf8>, 'MobileNetV2': <function MobileNetV2 at 0x7f6d46cad048>, 'MobileNetV3Large': <function MobileNetV3Large at 0x7f6d46cad8c8>, 'MobileNetV3Small': <

In [ ]:
# Loop over each model available in Keras
model_benchmarks = {'model_name': [], 'num_model_params': [], 'validation_loss': [], 'validation_accuracy': []}
print(model_benchmarks)

{'model_name': [], 'num_model_params': [], 'validation_loss': [], 'validation_accuracy': []}


In [ ]:
# 'Xception'
# 'MobileNetV3Large'
# 'DenseNet121'
# 'EfficientNetB0'
# 'EfficientNetB1'
# 'EfficientNetB2'
# 'VGG16'
# 'ResNet152'
# 'NASNetLarge'

model_name = "ResNet152"
model = model_dictionary[model_name]

pre_trained_model = model(
    include_top=False,
    weights='imagenet', 
    pooling='avg', 
    input_shape=(input_dim, input_dim, 3)
)

In [ ]:
import wandb
from wandb.keras import WandbCallback
from datetime import datetime as dt

clf_model = tf.keras.models.Sequential([layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),])
clf_model.add(pre_trained_model)
clf_model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
clf_model.compile(loss='categorical_crossentropy',optimizer="sgd",metrics=['accuracy'])

# callback = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss',
#     min_delta=0,
#     patience=5,
#     verbose=1,
#     mode='auto',
#     baseline=None,
#     restore_best_weights=True
# )

run = wandb.init(project='fireClassification', name="other")

history = clf_model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    steps_per_epoch=len(train_ds),
    callbacks = [WandbCallback()]
)

#save the model
results_path = "./"
clf_model.save(os.path.join(results_path, model_name+".h5"))

In [ ]:
run.finish()